In [1]:
import os, glob, re, json, math
from datetime import date, datetime, timedelta
from bs4 import BeautifulSoup
import numpy as np, pandas as pd
import requests, urllib.request, urllib.parse
from hunnae_naver_scraper import naver_news_scraper, naver_commentlist_scraper, naver_childcommentlist_scraper, naver_idhistory_scraper
from hunnae_daum_scraper import daum_news_scraper, daum_commentlist_scraper, daum_childcommentlist_scraper, daum_idhistory_scraper

# 1. 네이버 크롤러

In [2]:
url_naver = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=422&aid=0000450251'

### 1-1. 기사 정보 수집 : naver_news_scraper

In [3]:
news_dict_naver = naver_news_scraper(url_naver)
news_dict_naver

{'oid': '422',
 'aid': '0000450251',
 'title': '與 "안보, 정쟁화 말라" 野 "군, 정치의 포로"',
 'date': Timestamp('2020-10-01 14:22:00'),
 'summary': None,
 'body': '동영상 뉴스 추석이자 국군의날인 오늘(1일) 여야는 코로나19 방역과 안보의 중요성을 강조하면서도 서로를 향해 날을 세웠습니다. 더불어민주당 허영 대변인은 논평에서 국민의힘을 겨냥해 정치공학 셈법으로 국가안보를 정쟁화하지 말라고 비판했습니다. 허 대변인은 또한 개천절을 위기의 날로 바꾸려는 세력이 있다며, 좌시하지 않겠다고 강조했습니다. 반면에 국민의힘 김은혜 대변인은 논평에서 군이 공무원 피격 사건에서 무기력한 모습을 보이는 등 정치의 포로가 됐다고 주장했습니다. 그러면서 북한 대변인, 법무장관 비서를 자임하는 군은 존재 이유가 없다고 비판했습니다. 연합뉴스TV 기사문의 및 제보 : 카톡/라인 jebo23',
 'category': '정치',
 'press': '연합뉴스TV',
 'numcomment': 15}

### 1-2. 기사 댓글 json 수집  : naver_commentlist_scraper

In [4]:
comments_json_naver = naver_commentlist_scraper(url_naver)
comments_json_naver[0]

{'ticket': 'news',
 'objectId': 'news422,0000450251',
 'categoryId': '*',
 'templateId': 'default_politics',
 'commentNo': 2265322013,
 'parentCommentNo': 2265322013,
 'replyLevel': 1,
 'replyCount': 0,
 'replyAllCount': 0,
 'replyPreviewNo': None,
 'replyList': None,
 'imageCount': 0,
 'imageList': None,
 'imagePathList': None,
 'imageWidthList': None,
 'imageHeightList': None,
 'commentType': 'txt',
 'stickerId': None,
 'sticker': None,
 'sortValue': 1601529924036,
 'contents': '참으로 전세계에 얼굴이 뜨겁다. 어찌 이리 질낮은 의원들만 골라 뽑았는지...',
 'userIdNo': '1BNSu',
 'exposedUserIp': None,
 'lang': 'ko',
 'country': 'KR',
 'idType': 'naver',
 'idProvider': 'naver',
 'userName': 'duda****',
 'userProfileImage': '',
 'profileType': 'naver',
 'modTime': '2020-10-01T14:25:24+0900',
 'modTimeGmt': '2020-10-01T05:25:24+0000',
 'regTime': '2020-10-01T14:25:24+0900',
 'regTimeGmt': '2020-10-01T05:25:24+0000',
 'sympathyCount': 6,
 'antipathyCount': 0,
 'userBlind': False,
 'hideReplyButton': False,
 'status': 0,

### 1-3. 기사 '대'댓글 json 수집  : naver_childcommentlist_scraper

In [5]:
childcomments_json_naver = naver_childcommentlist_scraper(url_naver, comments_json_naver)
childcomments_json_naver[0]

{'ticket': 'news',
 'objectId': 'news422,0000450251',
 'categoryId': '*',
 'templateId': 'default_politics',
 'commentNo': 2265346393,
 'parentCommentNo': 2265344693,
 'replyLevel': 2,
 'replyCount': 0,
 'replyAllCount': 0,
 'replyPreviewNo': 1,
 'replyList': None,
 'imageCount': 0,
 'imageList': None,
 'imagePathList': None,
 'imageWidthList': None,
 'imageHeightList': None,
 'commentType': 'txt',
 'stickerId': None,
 'sticker': None,
 'sortValue': 1601532076625,
 'contents': '정권의개가 돼서 댓글이나 쳐달고 반인류적 쿠테타나 모의했던 박근혜정부시절 그게 바로 보수의 민낯이다',
 'userIdNo': '9ArRB',
 'exposedUserIp': None,
 'lang': 'ko',
 'country': 'KR',
 'idType': 'naver',
 'idProvider': 'naver',
 'userName': 'ohsg****',
 'userProfileImage': '',
 'profileType': 'naver',
 'modTime': '2020-10-01T15:01:17+0900',
 'modTimeGmt': '2020-10-01T06:01:17+0000',
 'regTime': '2020-10-01T15:01:17+0900',
 'regTimeGmt': '2020-10-01T06:01:17+0000',
 'sympathyCount': 1,
 'antipathyCount': 0,
 'userBlind': False,
 'hideReplyButton': False,
 'st

### 1-4. 유저 댓글 이력 json 수집 : naver_idhistory_scraper

In [6]:
comment_no1_naver = comments_json_naver[0]['commentNo']
comment_no2_naver = childcomments_json_naver[0]['commentNo']

In [7]:
idhistory_no1_naver = naver_idhistory_scraper(url_naver, comment_no1_naver)
idhistory_no2_naver = naver_idhistory_scraper(url_naver, comment_no2_naver)

# 2. 다음 크롤러

In [2]:
url_daum = 'https://news.v.daum.net/v/20201008211607146'
#url_daum = 'https://news.v.daum.net/v/20200908151019957'

### 2-1. 기사 정보 수집 : daum_news_scraper

In [3]:
news_dict_daum = daum_news_scraper(url_daum)
news_dict_daum

{'title': '과세대상 늘려도 9만명.."주가폭락 우려 현실성 없다" 지적',
 'postkey': '20201008211607146',
 'date': '2020.10.08',
 'content': '내년부터 주식 양도차익 과세 대상인 대주주 기준을 종목당 10억원에서 3억원으로 낮추는 자본소득 과세 강화 방안이 미뤄질 가능성이 커지고 있다. 기획재정부가 예정대로 추진하겠다는 뜻을 거듭 밝히고 있지만, 개인투자자들의 반발에 여당이 2년 유예를 공식화하고 나섰고 야당도 적극 동조하고 있기 때문이다. 하지만 이는 자본소득에 대한 과세 강화로 조세 형평성을 높이겠다는 문재인 정부의 국정 방향과 어긋나는 것이어서 논란이 예상된다. 김태년 더불어민주당 원내대표는 8일 국정감사 대책회의에서 “2년 뒤면 양도소득세가 전면 도입되는 만큼 대주주 요건 완화는 달라진 사정에 맞춰 재검토가 필요하다. 정책은 일관성이 있어야 하지만 상황 변화와 현장 수용성도 그에 못지않게 중요하다”며 “민주당은 여론을 충분히 수렴한 뒤에 조속한 시일 내에 당정협의를 통해 관련 정책을 결정하겠다”고 말했다. 홍남기 경제부총리 겸 기획재정부 장관이 7∼8일 이틀에 걸친 기재부 국감에서 대주주 기준 완화 방침에 변화가 없다는 점을 분명히 한 상황에서 여당 원내대표가 이를 공개적으로 반박한 것이다. 대주주 기준 완화 유예는 여당뿐 아니라 제1야당인 국민의힘도 적극적으로 동조하고 있어, 정부나 청와대가 끝까지 반대할 경우 여야 합의로 입법을 통해 과세 대상 확대 방안이 무산될 수도 있다. 실제 추경호 국민의힘 의원은 이날 기재부 국감에서 “여당 의원들과 의견이 같다. 법은 국회에서 제정하는 것이니 기재부 의견은 참고하고 여야가 뜻을 모으면 (현행 기준 유지가) 가능할 것”이라고 말했다. 주식 대량 보유자이자 양도차익 과세 대상인 대주주의 정의는 소득세법 시행령에 담겨 있는데, 해마다 그 대상이 확대돼 왔다. 2016년 25억원이던 기준이 2018년 15억원, 2020년 10억원으로 낮아졌다. 내년엔

### 2-2. 기사 댓글 json 수집 : daum_commentlist_scraper

In [4]:
comments_json_daum = daum_commentlist_scraper(url_daum)
comments_json_daum[0]

{'id': 590600173,
 'userId': -2026478,
 'postId': 149184951,
 'forumId': -99,
 'parentId': 0,
 'type': 'COMMENT',
 'status': 'S',
 'flags': 256,
 'rating': 0,
 'content': '한.종.목.에 3억원 가진 사람이 얼마나 많다고 난리들이냐?',
 'createdAt': '2020-10-08T21:33:17+0900',
 'updatedAt': '2020-10-08T21:33:17+0900',
 'childCount': 2,
 'likeCount': 1,
 'dislikeCount': 2,
 'recommendCount': -1,
 'screenedByKeeper': False,
 'user': {'id': -2026478,
  'status': 'S',
  'type': 'USER',
  'flags': 0,
  'icon': 'https://t1.daumcdn.net/profile/7CFRwQdCR5M0',
  'url': '',
  'username': 'DAUM:8vb8',
  'roles': 'ROLE_USER,ROLE_DAUM,ROLE_IDENTIFIED',
  'providerId': 'DAUM',
  'providerUserId': '8vb8',
  'displayName': 'wgfrit',
  'description': '',
  'commentCount': 262}}

### 2-3. 기사 '대'댓글 json 수집  : daum_childcommentlist_scraper

In [5]:
childcomments_json_daum = daum_childcommentlist_scraper(comments_json_daum)
childcomments_json_daum[0]

{'id': 590615508,
 'userId': -5660204,
 'postId': 149184951,
 'forumId': -99,
 'parentId': 590600173,
 'type': 'COMMENT',
 'status': 'S',
 'flags': 256,
 'rating': 0,
 'content': '주식안하지?',
 'createdAt': '2020-10-08T22:23:39+0900',
 'updatedAt': '2020-10-08T22:23:39+0900',
 'childCount': 0,
 'likeCount': 0,
 'dislikeCount': 0,
 'recommendCount': 0,
 'screenedByKeeper': False,
 'post': {'id': 149184951,
  'forumId': -99,
  'userId': 0,
  'postKey': '20201008211607146',
  'type': 'AUTO',
  'status': 'S',
  'flags': 0,
  'createdAt': '2020-10-08T21:16:27+0900',
  'updatedAt': '2020-10-08T21:16:27+0900',
  'title': '대주주 늘려도 9만명.."주가폭락 우려 현실성 없다" 지적',
  'url': 'https://news.v.daum.net/v/20201008211607146',
  'icon': 'https://img1.daumcdn.net/thumb/S1200x630/?fname=https://t1.daumcdn.net/news/202010/08/hani/20201008211607860grvb.jpg',
  'commentCount': 8,
  'childCount': 3,
  'officialCount': 0},
 'user': {'id': -5660204,
  'status': 'S',
  'type': 'USER',
  'flags': 0,
  'icon': 'https://t1.

### 2-4. 유저 댓글 이력 json 수집 : daum_idhistory_scraper

In [6]:
comment_no1_daum = comments_json_daum[0]['userId']
comment_no2_daum = childcomments_json_daum[0]['userId']

In [7]:
idhistory_no1_daum = daum_idhistory_scraper(comment_no1_daum)
idhistory_no2_daum = daum_idhistory_scraper(comment_no2_daum)